<h1 style="text-align: center;">Llama3 fine-tuning on IOB data</h1>

# Libraries

In [ ]:
#Connect to Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
colab_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', colab_path)
sys.path.insert(0,colab_path)


Mounted at /content/drive


In [ ]:
#Libraries
!pip install -q "torch==2.2.2" tensorboard

!pip install -q --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.2.2 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.0 MB

Accessing Llama3 needs permission from Meta and needs to be requested, after receiving the permission, login to HuggingFace is required to use the access token.

In [ ]:
#Login to huggingface to access the model
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


# Dataset

This data has been annotated manually and formulated on the IOB schema to define named entity tags based on the former data annotation at the segment level.

In [ ]:
#Import sentence dataframe
df = pd.read_csv(r"/content/drive/MyDrive/Final Project/Field Project/NER - IOB/Input_df.csv")

In [ ]:
df.head()

,Doc_Name,Unnamed: 1,word,tag,Tfidf_lemma,normalized_POS,normalized_Dependency_pars,Paragraph_place
0,A Rydberg 1,0,Rydberg,B,0.168232,0.6875,0.318182,A Rydberg 1
1,A Rydberg 1,1,atomic,I,0.056077,0.0000,0.136364,A Rydberg 1
2,A Rydberg 1,2,sensors,I,0.056077,0.4375,0.659091,A Rydberg 1
3,A Rydberg 1,3,and,I,0.224309,0.2500,0.272727,A Rydberg 1
4,A Rydberg 1,4,receivers,I,0.056077,0.4375,0.340909,A Rydberg 1


In [ ]:
from sklearn.model_selection import train_test_split

#Create Train/Validation/Test set and ensure split point is not in the middle of a sequence
grouped = df.groupby('Paragraph_place')

sizes = grouped.size().cumsum() #cumulative size

train_size = 0.7 * sizes.iloc[-1]
val_size = 0.2 * train_size
test_size = sizes.iloc[-1] - train_size

#Train, test index based on 'Paragraph_place' column
train_idx = sizes[sizes <= train_size].index
test_idx = sizes[sizes > train_size].index


train_data_with_val = df[df['Paragraph_place'].isin(train_idx)]
test_data = df[df['Paragraph_place'].isin(test_idx)]

#Validation set
train_sizes = train_data_with_val.groupby('Paragraph_place').size().cumsum()
val_idx = train_sizes[train_sizes <= val_size].index
final_train_idx = train_sizes[train_sizes > val_size].index

val_data = train_data_with_val[train_data_with_val['Paragraph_place'].isin(val_idx)]
train_data = train_data_with_val[train_data_with_val['Paragraph_place'].isin(final_train_idx)]


In [ ]:
train_data.head()

,Doc_Name,Unnamed: 1,word,tag,Tfidf_lemma,normalized_POS,normalized_Dependency_pars,Paragraph_place
14134,Firmicutes 23,81493,qPCR,O,0.204598,1.0000,0.681818,Firmicutes 23
14135,Firmicutes 23,81494,was,O,0.409197,0.1875,0.227273,Firmicutes 23
14136,Firmicutes 23,81495,performed,O,0.136399,0.9375,0.000000,Firmicutes 23
14137,Firmicutes 23,81496,using,O,0.204598,0.9375,0.022727,Firmicutes 23
14138,Firmicutes 23,81497,an,O,0.068199,0.3125,0.454545,Firmicutes 23


In [ ]:
columns_to_drop = ['Doc_Name', 'Unnamed: 1', 'Tfidf_lemma', 'normalized_POS', 'normalized_Dependency_pars', 'Paragraph_place']

In [ ]:
train_data = train_data.drop(columns=columns_to_drop)
val_data = val_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)


In [ ]:
X_train = train_data
X_val = val_data
X_test = test_data

In [ ]:
#Finding noise in the data
unique_tags_tarin = X_train['tag'].unique()
print("Unique tags in train set:", unique_tags_tarin)
unique_tags_test = X_test['tag'].unique()
print("Unique tags in test set:", unique_tags_test)
unique_tags_val = X_val['tag'].unique()
print("Unique tags in validation set:", unique_tags_val)

Unique tags in train set: ['O' 'B' 'I' 'P' 'o' ',']
Unique tags in test set: ['O' 'B' 'I' 'o']
Unique tags in validation set: ['B' 'I' 'O' '[']


As some data was labelled wrong, we replaced them with "O".

In [ ]:
def clean_tags(dataset):

    valid_labels = ['O', 'B', 'I']

    dataset['tag'] = dataset['tag'].apply(lambda tag: tag if tag in valid_labels else 'O')

    return dataset

In [ ]:
X_train = clean_tags(X_train)
X_test = clean_tags(X_test)
X_val = clean_tags(X_val)


In [ ]:
unique_tags_tarin = X_train['tag'].unique()
print("Unique tags after cleaning in train set:", unique_tags_tarin)
unique_tags_test = X_test['tag'].unique()
print("Unique tags after cleaning in train set:", unique_tags_test)
unique_tags_val = X_val['tag'].unique()
print("Unique tags after cleaning in train set:", unique_tags_val)

Unique tags after cleaning in train set: ['O' 'B' 'I']
Unique tags after cleaning in train set: ['O' 'B' 'I']
Unique tags after cleaning in train set: ['B' 'I' 'O']


In [ ]:
#Dict for mapping labels to numbers
tag_to_num = {'O': 0, 'B': 1, 'I': 2}

X_train['tag_num'] = X_train['tag'].map(tag_to_num)
X_test['tag_num'] = X_test['tag'].map(tag_to_num)
X_val['tag_num'] = X_val['tag'].map(tag_to_num)


In [ ]:
from datasets import Dataset, DatasetDict
# pandas to Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(X_train.drop(columns=['tag']).reset_index(drop=True))
dataset_val = Dataset.from_pandas(X_val.drop(columns=['tag']).reset_index(drop=True))
dataset_test = Dataset.from_pandas(X_test.drop(columns=['tag']).reset_index(drop=True))


In [ ]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['word', 'tag_num'],
        num_rows: 56879
    })
    val: Dataset({
        features: ['word', 'tag_num'],
        num_rows: 14134
    })
    test: Dataset({
        features: ['word', 'tag_num'],
        num_rows: 30554
    })
})

In [ ]:
X_train.tag.value_counts(normalize=True)


,proportion
tag,
O,0.658767
I,0.308427
B,0.032806


We defined class weight as the data is not uniformly distributed.

In [ ]:
class_weights=(1/X_train.tag.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

tensor([0.8649, 0.0920, 0.0431])

# Model Configuration

We used Llama3 8B for finetuning, as the 70B caused severe memory issues even with Google Colab's L4 GPU with High RAM.

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B" #Model we used (Other options is Meta-Llama-3-70B)

In [ ]:
#quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, #4-bit precision to reduce memory
    bnb_4bit_quant_type = 'nf4', #non-linear quantization
    bnb_4bit_use_double_quant = True, #double quantization (to imrove performance)
    bnb_4bit_compute_dtype = torch.bfloat16
)


In [ ]:
#LoRA
lora_config = LoraConfig(
    r = 64, # dimension
    lora_alpha = 16,
    target_modules = [
        'q_proj', 'k_proj', 'v_proj', 'o_proj',  # Self-attention projections
        'gate_proj', 'up_proj', 'down_proj'     # MLP projections
    ], #all linear layers
    lora_dropout = 0.1,
    bias = 'none',
    task_type = 'SEQ_CLS' #sequence classification
)

Quantization of LoRA is usually called QLoRA. Based on [TextMine](https://textmine.com/post/an-introduction-to-llm-quantization) LLM quantization is an approach to processing large amounts of language data, which converts a continuous data set to a discrete data set – ultimately minimising the number of bits needed to display the signal.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=3 #"I", "O", "B"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model = get_peft_model(model, lora_config) #Parameter-Efficient Fine-Tuning, applying Low-Rank Adaptation (LoRa)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id # "eos" stands for end-of-sequence
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id #match model's- and Tokenizer's pad_token_id
model.config.use_cache = False # True can cause memory issues
model.config.pretraining_tp = 1 #tensor parallelism (1 = single-device training)

In [ ]:
#Check the model before fine-tunning
words = X_test.word.tolist()

batch_size = 128

all_outputs = []

for i in range(0, len(words), batch_size):
    batch_of_words = words[i:i + batch_size]

    inputs = tokenizer(batch_of_words, return_tensors="pt", padding=True, truncation=True, max_length=256)

    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])



In [ ]:
#Calculate output and move them to CPU
outputs = torch.cat(all_outputs, dim=0)
X_test['predictions_before_training']=outputs.argmax(axis=1).cpu().numpy()


In [ ]:
# Dict to map numbers to classes
num_to_tag = { 0: 'O', 1:'B', 2:'I'}

In [ ]:
X_test['predictions_before_training']=X_test['predictions_before_training'].map(num_to_tag)
X_test

,word,tag,tag_num,predictions_before_training
71013,The,O,0,O
71014,reproducibility,O,0,I
71015,and,O,0,I
71016,precision,O,0,I
71017,of,O,0,O
...,...,...,...,...
101562,robust,I,2,I
101563,municipal,I,2,O
101564,waste,I,2,I
101565,systems,I,2,I


In [ ]:
# F1-score calculation for Llama3 before fine-tunning
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

y_test = X_test.tag
y_pred = X_test.predictions_before_training

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("F-1 Score:", f1_score(y_test, y_pred, average='weighted'))

Confusion Matrix:
[[   18   589   286]
 [  205  6717  1912]
 [  435 14274  6118]]

Classification Report:
              precision    recall  f1-score   support

           B       0.03      0.02      0.02       893
           I       0.31      0.76      0.44      8834
           O       0.74      0.29      0.42     20827

    accuracy                           0.42     30554
   macro avg       0.36      0.36      0.29     30554
weighted avg       0.59      0.42      0.41     30554

Accuracy Score: 0.4206650520390129
F-1 Score: 0.41458346404449553


In [ ]:
MAX_LEN = 16 #Max length of each input

def llama_preprocessing_function(df):
    return tokenizer(df['word'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("tag_num", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/56879 [00:00<?, ? examples/s]

Map:   0%|          | 0/14134 [00:00<?, ? examples/s]

Map:   0%|          | 0/30554 [00:00<?, ? examples/s]

In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer) #Padding


In [ ]:
#Metrics
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1_score': f1_score(labels, predictions, average='weighted')
    }


In [ ]:
#Training Arguments
training_args = TrainingArguments(
    output_dir = 'IOB_SEQ_classification',
    learning_rate = 2e-4,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=8, #accumulate gradients to avoid memory issues
    gradient_checkpointing=True,
    num_train_epochs = 5,
    weight_decay = 0.01,
    logging_steps=10, #Number of update steps between two logs
    #max_steps=1000,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    fp16=True,
    load_best_model_at_end = True
)

In [ ]:
from transformers import Trainer, EarlyStoppingCallback, get_cosine_schedule_with_warmup
import torch

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# learning rate scheduler (Cosine)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=200,
    num_training_steps=len(tokenized_datasets['train']) // training_args.train_batch_size * training_args.num_train_epochs
)

In [ ]:
#Custom trainer to effect weights as classess are not uniformly distributed
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):

        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=collate_fn,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], #Early stopping
    compute_metrics=compute_metrics,
    class_weights=class_weights
)

<ipython-input-36-1bea759fa04e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


# Training and result

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
0,0.143300,0.845989,0.625442,0.481319
1,0.139400,0.686157,0.625442,0.481319
2,0.134600,0.804968,0.625371,0.481306
4,0.132800,0.807845,0.625371,0.481306


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

In [ ]:
# Prediction function
def make_predictions(model,df_test):

  words = df_test.word.tolist()

  batch_size = 128

  all_outputs = []

  for i in range(0, len(words), batch_size):
      batch_of_words = words[i:i + batch_size]

      inputs = tokenizer(batch_of_words, return_tensors="pt", padding=True, truncation=True, max_length=256)

      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions_finetune']=final_outputs.argmax(axis=1).cpu().numpy()


In [ ]:
make_predictions(model,X_test)

In [ ]:
X_test.head()

,word,tag,tag_num,predictions_finetune
71013,The,O,0,0
71014,reproducibility,O,0,0
71015,and,O,0,0
71016,precision,O,0,0
71017,of,O,0,0


In [ ]:
#to_csv
X_test.to_csv(r"/content/drive/MyDrive/Final Project/Field Project/IOB-Llama3-prediction.csv")

In [ ]:
# F1-score after fine-tunning
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

y_test = X_test.tag_num
y_pred = X_test.predictions_finetune

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("F-1 Score:", f1_score(y_test, y_pred, average='weighted'))



Confusion Matrix:
[[20827     0     0]
 [  893     0     0]
 [ 8834     0     0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81     20827
           1       0.00      0.00      0.00       893
           2       0.00      0.00      0.00      8834

    accuracy                           0.68     30554
   macro avg       0.23      0.33      0.27     30554
weighted avg       0.46      0.68      0.55     30554

Accuracy Score: 0.6816456110492898
F-1 Score: 0.5526024460918844


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
metrics = train_result.metrics
max_train_samples = len(X_train)
metrics["train_samples"] = min(max_train_samples, len(X_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        4.0
  total_flos               = 38729528GF
  train_loss               =     0.2666
  train_runtime            = 3:30:36.48
  train_samples            =      56879
  train_samples_per_second =     22.506
  train_steps_per_second   =      0.088


In [ ]:
trainer.save_model("/content/drive/My Drive/Llama3/IOB-llama3/IOB-llama3_trainer") #save trainer

In [ ]:
model.save_pretrained("/content/drive/My Drive/Llama3/IOB-llama3/IOB-llama3_model.pt") #save model

In [ ]:
!pip freeze > requirements.txt